In [1]:
from google.cloud import storage, bigquery
import os


In [18]:
project_id = os.getenv("GBQ_PROJECT_ID")
bucket_nm = os.getenv("GCS_BUCKET")

In [23]:
def get_gcs_uris_to_load(bucket_nm, prefix) -> list:
    gcs_client = storage.Client(project_id)
    bucket = gcs_client.get_bucket(bucket_nm)
    blobs = bucket.list_blobs(prefix=prefix)

    # only grab parquet files
    gcs_uris = [blob.public_url for blob in blobs if blob.name.endswith('.parquet')]

    return gcs_uris

In [24]:
def load_bq_from_gcs(gcs_uris, dataset_nm, table_nm) -> None:
    table_id = f"{project_id}.{dataset_nm}.{table_nm}"
    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.PARQUET
    )

    bq_client = bigquery.Client(project_id)
    job = bq_client.load_table_from_uri(
        gcs_uris,
        table_id,
        job_config=job_config,
    ).result()


In [25]:
prefix = "test_files/par"
gcs_uris = get_gcs_uris_to_load(bucket_nm, prefix)

In [26]:
load_bq_from_gcs(gcs_uris, "sandbox", "test_internal_from_ext")

In [3]:
def upload_file_to_gcs(local_file_path, gcs_bucket_name, gcs_key_path):
    
    client = storage.Client()
    bucket = client.get_bucket(gcs_bucket_name) 
    blob = bucket.blob(gcs_key_path)
    blob.upload_from_filename(local_file_path)
    print("file {0} uploaded successfully".format(os.path.basename(local_file_path)))


In [2]:
def upload_file_to_gcs_v2(local_file_path):
    gcs_bucket_name = "data-mattm-test-sbx"
    gcs_key_path = "test_files/"+os.path.basename(local_file_path)
    client = storage.Client()
    bucket = client.get_bucket(gcs_bucket_name) 
    blob = bucket.blob(gcs_key_path)
    blob.upload_from_filename(local_file_path)
    print("file {0} uploaded successfully".format(os.path.basename(local_file_path)))

In [5]:
import concurrent.futures
import glob

local_folder_path = os.path.expanduser("~/test_dummy_data/gcs/*.csv")
gcs_bucket_name = "data-mattm-test-sbx"

csv_files = glob.glob(local_folder_path)

In [6]:
gcs_bucket_name = "data-mattm-test-sbx"
for local_file_path in csv_files:
    gcs_key_path = "test_files/"+os.path.basename(local_file_path)
    print(gcs_key_path)
    upload_file_to_gcs(local_file_path, gcs_bucket_name, gcs_key_path)

    #futures.append(executor.submit(upload_file_to_gcs, local_file_path, gcs_bucket_name, gcs_key_path))

test_files/test8.csv
file test8.csv uploaded successfully
test_files/test9.csv
file test9.csv uploaded successfully
test_files/test4.csv
file test4.csv uploaded successfully
test_files/test5.csv
file test5.csv uploaded successfully
test_files/test7.csv
file test7.csv uploaded successfully
test_files/test6.csv
file test6.csv uploaded successfully
test_files/test2.csv
file test2.csv uploaded successfully
test_files/test3.csv
file test3.csv uploaded successfully
test_files/test1.csv
file test1.csv uploaded successfully
test_files/test10.csv
file test10.csv uploaded successfully


In [12]:
import concurrent.futures
import glob

local_folder_path = os.path.expanduser("~/test_dummy_data/gcs/*.csv")
gcs_bucket_name = "data-mattm-test-sbx"

csv_files = glob.glob(local_folder_path)

with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    executor.map(upload_file_to_gcs_v2, csv_files)

# with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
#     futures = []
#     for local_file_path in csv_files:
#         gcs_key_path = "test_files/"+os.path.basename(local_file_path)
#         futures.append(executor.submit(upload_file_to_gcs, local_file_path, gcs_bucket_name, gcs_key_path))

file test8.csv uploaded successfullyfile test5.csv uploaded successfully
file test6.csv uploaded successfully
file test2.csv uploaded successfully
file test7.csv uploaded successfully

file test1.csv uploaded successfully
file test4.csv uploaded successfully
file test3.csv uploaded successfully
file test10.csv uploaded successfully
file test9.csv uploaded successfully


In [17]:
def upload_file_to_gcs_v2(local_file_path):
    gcs_bucket_name = "data-mattm-test-sbx"
    gcs_key_path = "test_files/"+os.path.basename(local_file_path)
    client = storage.Client()
    bucket = client.get_bucket(gcs_bucket_name) 
    blob = bucket.blob(gcs_key_path)
    blob.upload_from_filename(local_file_path)
    print("file {0} uploaded successfully".format(os.path.basename(local_file_path)))


In [18]:
import concurrent.futures
import glob


local_folder_path = os.path.expanduser("~/test_dummy_data/gcs/*.csv")
gcs_bucket_name = "data-mattm-test-sbx"

csv_files = glob.glob(local_folder_path)

with concurrent.futures.ProcessPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(upload_file_to_gcs_v2, local_file_path) for local_file_path in csv_files]

    for future in concurrent.futures.as_completed(futures):
        try:
            result = future.result()
        except Exception as e:
            print(f"Error: {e}")

Error: A process in the process pool was terminated abruptly while the future was running or pending.
Error: A process in the process pool was terminated abruptly while the future was running or pending.
Error: A process in the process pool was terminated abruptly while the future was running or pending.
Error: A process in the process pool was terminated abruptly while the future was running or pending.
Error: A process in the process pool was terminated abruptly while the future was running or pending.
Error: A process in the process pool was terminated abruptly while the future was running or pending.
Error: A process in the process pool was terminated abruptly while the future was running or pending.
Error: A process in the process pool was terminated abruptly while the future was running or pending.
Error: A process in the process pool was terminated abruptly while the future was running or pending.
Error: A process in the process pool was terminated abruptly while the future was 

Process SpawnProcess-45:
Process SpawnProcess-43:
Process SpawnProcess-41:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process SpawnProcess-42:
Traceback (most recent call last):
Process SpawnProcess-46:
  File "/opt/homebrew/Cellar/python@3.11/3.11.5/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/homebrew/Cellar/python@3.11/3.11.5/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/homebrew/Cellar/python@3.11/3.11.5/Frameworks/Python.framework/Versions/3.11/lib/python3.11/concurrent/futures/process.py", line 244, in _process_worker
    call_item = call_queue.get(block=True)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.11/3.11.5/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multipro